In [14]:
import sys
import sqlite3

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import RFECV, RFE
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATABASE_PATH = "./database/database.db"

conn = sqlite3.connect(DATABASE_PATH)

data = pd.read_sql(sql='SELECT * FROM data', con=conn)

conn.close()

In [16]:
di = 'SPEI'
di_scale = 6

selected_columns = [
    'Station_Name', 'Station_ID',
    'Station_Latitude', 'Station_Longitude', 'Station_Elevation',
    'Date',
    f'{di}_{di_scale}',
    'ERA5_Precipitation',
    'GPM_Precipitation',
    'TRMM_Precipitation',
    'TERRACLIMATE_Precipitation',
    'PERSIANNCDR_Precipitation',
    f'GPM_{di}_{di_scale}',
    f'ERA5_{di}_{di_scale}',
    'PET_MOD16A2GF',
    'NDVI', 'EVI',
    'LSTDay', 'LSTNight', 'LST',
    'PCI_GPM', 'VCI', 'TCI', 'VHI', 'CI_GPM',
]

In [17]:
df = data.filter(items=selected_columns)
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Station_ID'] = df['Station_ID'].astype('category')
df['Year'] = df['Year'].astype('category')
df['Month'] = df['Month'].astype('category')

In [18]:
selected_stations = ['Ramsar', 'Nowshahr']
# selected_stations = ['Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht']
# selected_stations = ['Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']

start_date = '2006-09'
end_date = '2023-10'

df = df.query("Station_Name in @selected_stations and Date >= @start_date and Date < @end_date")

In [19]:
df.dropna(inplace=True)
df.sort_values(by=['Station_ID', 'Year', 'Month'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [20]:
df[f'{di}_{di_scale}_Class'] = pd.cut(df[f'{di}_{di_scale}'], bins=[-10, -2, -1.5, -1, 1, 1.5, 2, 10], labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])
df[f'{di}_{di_scale}_Class'] = df[f'{di}_{di_scale}_Class'].astype('category')

In [21]:
cols_to_lag = ['GPM_Precipitation', 'PET_MOD16A2GF', 'NDVI', 'EVI', 'VHI', 'LST', 'PCI_GPM', 'TCI', 'VHI', 'CI_GPM']
for lag in range(1, 13):
    for col in cols_to_lag:
        df[f'{col}_lag_{lag}'] = df.groupby('Station_ID', observed=False)[col].shift(lag)

/tmp/ipykernel_10540/547769836.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df.groupby('Station_ID', observed=False)[col].shift(lag)
/tmp/ipykernel_10540/547769836.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df.groupby('Station_ID', observed=False)[col].shift(lag)
/tmp/ipykernel_10540/547769836.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [22]:
for col in cols_to_lag:
    df[f'{col}_roll_mean_3'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
    df[f'{col}_roll_std_3'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=3, min_periods=1).std())
    df[f'{col}_roll_mean_6'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=6, min_periods=1).mean())
    df[f'{col}_roll_std_6'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=6, min_periods=1).std())
    df[f'{col}_roll_mean_9'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=9, min_periods=1).mean())
    df[f'{col}_roll_std_9'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=9, min_periods=1).std())
    df[f'{col}_roll_mean_12'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=12, min_periods=1).mean())
    df[f'{col}_roll_std_12'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=12, min_periods=1).std())


/tmp/ipykernel_10540/1487553917.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll_mean_3'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
/tmp/ipykernel_10540/1487553917.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll_std_3'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=3, min_periods=1).std())
/tmp/ipykernel_10540/1487553917.py:4: PerformanceWarning: DataFrame is highly fragmented.  This 

In [23]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

X = df.drop(
    columns=[
        'Station_Name',
        'Station_ID',
        'Date',
        'Year',
        f'{di}_{di_scale}',
        f'{di}_{di_scale}_Class',
    ]
)

y_raw = df[f'{di}_{di_scale}_Class']

# y = df[f'{di}_{di_scale}_Class']

# Stacking Meta-Model

In [24]:
df['Month'] = df['Month'].astype('int')
df['month_sin'] = np.sin(2 * np.pi * df['Month']/12.0)
df['month_cos'] = np.cos(2 * np.pi * df['Month']/12.0)
df['Month'] = df['Month'].astype('category')
# X.drop(columns=['Month'], inplace=True)


# --- FIX: Encode string labels to integers ---
encoder = LabelEncoder()
y = encoder.fit_transform(y_raw)
print("\nClass labels encoded.")

# --- 3. Data Cleaning and Splitting ---
class_counts = pd.Series(y).value_counts()
classes_to_remove_indices = class_counts[class_counts < 2].index
if len(classes_to_remove_indices) > 0:
    # پیدا کردن اندیس های اصلی در دیتافریم
    original_indices_to_remove = df.loc[y_raw.isin(encoder.inverse_transform(classes_to_remove_indices))].index
    print(f"\nRemoving classes with fewer than 2 samples: {list(encoder.inverse_transform(classes_to_remove_indices))}")
    X = X.drop(original_indices_to_remove)
    y = np.delete(y, original_indices_to_remove) # y اکنون یک آرایه نامپای است
    print(f"New data shape after removing rare classes: {X.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# --- 4. Automatic Feature Selection ---
print("\n--- Performing Automatic Feature Selection with RFECV ---")
estimator = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
max_features_cap = 15

selector_cv = RFECV(estimator, step=1, cv=StratifiedKFold(3), scoring='f1_macro', min_features_to_select=5, n_jobs=-1)
selector_cv = selector_cv.fit(X_train, y_train)

optimal_n_features = selector_cv.n_features_
print(f"Optimal number of features found by RFECV: {optimal_n_features}")

if optimal_n_features > max_features_cap:
    print(f"Capping features to {max_features_cap}.")
    selector = RFE(estimator, n_features_to_select=max_features_cap, step=1)
    selector = selector.fit(X_train, y_train)
else:
    selector = selector_cv

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
selected_feature_names = X_train.columns[selector.support_] # نام ستون های انتخاب شده


# --- 5. Data Balancing ---
print("\nBalancing training data with SMOTE...")
min_class_size = pd.Series(y_train).value_counts().min()
k_neighbors = max(1, min_class_size - 1)
smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_selected, y_train)

# --- 6. Hyperparameter Tuning for Each Base Model ---
print("\n--- Hyperparameter Tuning for Base Models ---")

# تعریف مدل ها و فضای پارامترهای آنها
models_to_tune = {
    # 'rf': (RandomForestClassifier(random_state=42, n_jobs=-1), {
    #     'n_estimators': [100, 200, 400], 'max_depth': [10, 20, None], 'min_samples_split': [2, 5], 'class_weight': ['balanced']
    # }),
    # 'lgbm': (LGBMClassifier(random_state=42, n_jobs=-1), {
    #     'n_estimators': [400], 'learning_rate': [0.1], 'num_leaves': [40], 'class_weight': ['balanced']
    # }),
    # 'cat': (CatBoostClassifier(random_state=42, verbose=0), {
    #     'iterations': [200, 500], 'learning_rate': [0.05, 0.1], 'depth': [6, 8], 'auto_class_weights': ['Balanced']
    # }),
    'xgb': (XGBClassifier(random_state=42, eval_metric='mlogloss', n_jobs=-1), {
        'n_estimators': [100, 200, 400], 'learning_rate': [0.05, 0.1], 'max_depth': [5, 7]
    }),
    'svc': (make_pipeline(StandardScaler(), SVC(random_state=42, probability=True)), {
        'svc__C': [1, 10], 'svc__gamma': ['scale', 'auto'], 'svc__class_weight': ['balanced']
    }),
    'knn': (make_pipeline(StandardScaler(), KNeighborsClassifier(n_jobs=-1)), {
        'kneighborsclassifier__n_neighbors': [5, 7, 9]
    })
}

best_estimators = {}
for name, (model, params) in models_to_tune.items():
    print(f"\nTuning {name}...")
    search = RandomizedSearchCV(model, params, n_iter=10, cv=3, scoring='f1_macro', random_state=42, n_jobs=-1)
    search.fit(X_train_resampled, y_train_resampled)
    best_estimators[name] = search.best_estimator_
    print(f"Best params for {name}: {search.best_params_}")

# --- 7. Build and Train the Final Stacking Meta-Model ---
print("\n--- Building and Training the Final Stacking Model ---")

# استفاده از مدل های بهینه شده
estimators = [
    # ('rf', best_estimators['rf']), # {'n_estimators': 400, 'min_samples_split': 5, 'max_depth': 20, 'class_weight': 'balanced'}
    # ('lgbm', best_estimators['lgbm']),
    # ('cat', best_estimators['cat']), # {'learning_rate': 0.1, 'iterations': 500, 'depth': 6, 'auto_class_weights': 'Balanced'}
    ('xgb', best_estimators['xgb']),
    ('svc', best_estimators['svc']),
    ('knn', best_estimators['knn']),
    ('gnb', GaussianNB()) # Naive Bayes نیازی به بهینه سازی ندارد
]

# تعریف مدل نهایی برای ترکیب نتایج
final_estimator = LogisticRegression(random_state=42, class_weight='balanced')

stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=5,
    n_jobs=-1
)

stacking_model.fit(X_train_resampled, y_train_resampled)

# --- 8. Prediction and Evaluation ---
print("\n--- Evaluating the Final Model ---")
y_pred = stacking_model.predict(X_test_selected)

# برگرداندن برچسب های عددی به رشته ای برای گزارش
y_test_labels = encoder.inverse_transform(y_test)
y_pred_labels = encoder.inverse_transform(y_pred)

accuracy = accuracy_score(y_test_labels, y_pred_labels)
report = classification_report(y_test_labels, y_pred_labels)

print("\n--- Final Model Evaluation Metrics (Optimized Stacking Classifier) ---")
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)

# --- 9. Visualization ---
sns.set_theme(style="whitegrid")
cm = confusion_matrix(y_test_labels, y_pred_labels, labels=encoder.classes_)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title('Confusion Matrix (Optimized Stacking Meta-Model)', fontsize=16)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.show()




/tmp/ipykernel_10540/2755788743.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['month_sin'] = np.sin(2 * np.pi * df['Month']/12.0)
/tmp/ipykernel_10540/2755788743.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['month_cos'] = np.cos(2 * np.pi * df['Month']/12.0)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(



Class labels encoded.

Removing classes with fewer than 2 samples: ['EW']
New data shape after removing rare classes: (285, 202)

--- Performing Automatic Feature Selection with RFECV ---
Optimal number of features found by RFECV: 10

Balancing training data with SMOTE...

--- Hyperparameter Tuning for Base Models ---

Tuning xgb...


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/xgboost/core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/xgboost/sklearn.py", line 1641, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [0 2 3 4 5 6]
